In [ ]:
%matplotlib inline

!pip install keras
!pip install tensorflow  
!pip3 install opencv-python
!pip install pypng
!pip install matplotlib


from google.colab.patches import cv2_imshow
from skimage import data,filters
from PIL import ImageTk, Image


import cv2
import csv
import matplotlib.pyplot as plt
import matplotlib.pylab as plt
import matplotlib.cm as cm
import numpy as np
import pathlib
import PIL
import os,png,array
import pandas as pd
import time

In [ ]:
!unzip /content/drive/Shareddrives/Master/Analog/difficult_samples_filter.zip

In [ ]:
!unzip easy_samples.zip

In [ ]:
!wget https://github.com/ltdaovn/analog-meter-reading-openCV/raw/main/cnn_svhn_.model

In [ ]:
!unzip /content/drive/Shareddrives/Master/Analog/analog_manual_meter.zip

In [ ]:
!wget https://github.com/ltdaovn/analog-meter-reading-openCV/raw/main/analog_manual_meter_easy.xlsx

In [ ]:
from keras.models import load_model
classifier=load_model('cnn_svhn_.model')

row_list = []
index_counter = 0
resize_counter = 0

#directory="difficult_samples_filter/"
directory="easy_samples/"

lst = os.listdir(directory)
lst.sort()

for file in lst:
  #print(file)
  try:

    #if file.endswith(".jpg"): 
        #img = cv2.imread("difficult_samples_filter/" + file, 0)
    if file.endswith(".png"): 
        img = cv2.imread(directory + file, 0)

        if img is not None:
            #blur = cv2.GaussianBlur(grey, value, 0)
            img = cv2.GaussianBlur(img, (3,3), 0)

        #edges = cv2.Canny(img, 175, 200, apertureSize=3, L2gradient=False)
        edges = cv2.Canny(img, 50, 200, apertureSize=3, L2gradient=False)
        def auto_canny(image, sigma=0.33):

            v = np.median(image)
            lower = int(max(0, (1.0 - sigma) * v))
            upper = int(min(255, (1.0 + sigma) * v))
            edged = cv2.Canny(image, lower, upper, apertureSize=3, L2gradient=True)
            return edged

        #edges = auto_canny(img)
        if edges is None:
            break

        contours, hierarchy = cv2.findContours(edges.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        contours_dict = dict()
        for cont in contours:
            x, y, w, h = cv2.boundingRect(cont)
            area = cv2.contourArea(cont)
            if 10 < area and 10 < w and h > 5 :
                contours_dict[(x, y, w, h)] = cont

        contours_filtered = sorted(contours_dict.values(), key=cv2.boundingRect)
        blank_background = np.zeros_like(edges)
        img_contours = cv2.drawContours(blank_background, contours_filtered, -1, (255,255,255), thickness=2)

        def is_overlapping_horizontally(box1, box2):
            x1, _, w1, _ = box1
            x2, _, _, _ = box2
            if x1 > x2:
                return is_overlapping_horizontally(box2, box1)
            return (x2 - x1) < w1

        def merge(box1, box2):
            assert is_overlapping_horizontally(box1, box2)
            x1, y1, w1, h1 = box1
            x2, y2, w2, h2 = box2
            x = min(x1, x2)
            w = max(x1 + w1, x2 + w2) - x
            y = min(y1, y2)
            h = max(y1 + h1, y2 + h2) - y
            return (x, y, w, h)

        def windows(contours):
            boxes = []
            for cont in contours:
                box = cv2.boundingRect(cont)
                if not boxes:
                    boxes.append(box)
                else:
                    if is_overlapping_horizontally(boxes[-1], box):
                        last_box = boxes.pop()
                        merged_box = merge(box, last_box)
                        boxes.append(merged_box)
                    else:
                        boxes.append(box)
            return boxes

        boxes = windows(contours_filtered)
        len_boxes = len(boxes)

        for n in range(len_boxes):
          x, y, w, h = boxes[n]
          plt.axis('off')
          
          if y < 10:
              y = 10
          if x < 10:
              x = 10
              
          roi = img[y-10:y+h+5, x-10:x+w+5]
          roi = cv2.resize(roi,(32, 32),interpolation = cv2.INTER_AREA)
          a = str(n+1)
          cv2.imwrite('wmr'+a+'.png', roi)

        columnNames = list()

        for i in range(1024):
            pixel = 'pixel'
            pixel += str(i)
            columnNames.append(pixel)

        train_data = pd.DataFrame(columns = columnNames)

        for n in range(len_boxes):
            a = str(n+1)
            b='wmr'+a+'.png'
            img = Image.open(b)
            rawData = img.load()
            data = []
            for y in range(32):
                for x in range(32):
                    data.append(rawData[x,y])
            k = 0
            train_data.loc[0] = [data[k] for k in range(1024)]
            train_data = train_data.div(255)
            train_data.to_csv("train_converted"+a+".csv",index = False)


        w ={}
        for n in range(len_boxes):
            a = str(n+1)
            b = "train_converted"+a+".csv"
            x_pred = pd.read_csv(b)
            x_pred = x_pred.iloc[:,:].values.astype('float32')
            x_pred =x_pred.reshape(-1, 32, 32, 1)
            predictions=classifier.predict(x_pred)
            x = predictions[4]
            c = "predictions"+a
            w.update( {c : x.argmax()} )
        
        row = [file,  str(w.get("predictions1")) + 
               str(w.get("predictions2")) +  
               str(w.get("predictions3")) +
               str(w.get("predictions4")) +
               str(w.get("predictions5"))]

        row_list.append(row)

        print(file + " " + "Water meter: " +
            str(w.get("predictions1")) + 
            str(w.get("predictions2")) +  
            str(w.get("predictions3")) +
            str(w.get("predictions4")) +
            str(w.get("predictions5")) )

        continue
        
    else:
        continue

  except IndexError:
    if IndexError:
      index_counter += 1
      print(file + " " +'List index out of range error')
      print('index count: ', index_counter)
      pass

  '''except:
    row = [file, "Resize Error"]
    resize_counter += 1
    print('resize count: ', resize_counter)
    print(file + " " + 'resize error')'''

In [16]:
# Create a file for new predicted values, then write it
# update this csv each time you changed something in the code above 
# then follow these steps and calculate the accuracy, compare with other results
with open('analog_predicted_meter.csv', 'w', newline='') as file:
  writer = csv.writer(file)
  writer.writerows(row_list)
print(len(row_list))

1000


In [17]:
print("-------------------------------------")
print("resize counter error:", " ", resize_counter)
print("index counter error:" , " " , index_counter)
print("-------------------------------------")
read = 1000 - (int(resize_counter) + int(index_counter))
not_read = int(resize_counter) + int(index_counter)
print("reading: ", read)
print("not reading: ", not_read)
print("-------------------------------------")

-------------------------------------
resize counter error:   0
index counter error:   0
-------------------------------------
reading:  1000
not reading:  0
-------------------------------------


In [ ]:
read_value = pd.read_csv("analog_predicted_meter.csv")
read_value.columns = ["imageID", "read_value"]
read_value["imageID"] = read_value["imageID"].str.replace(r'.jpg$', '')
read_value["imageID"] = read_value["imageID"].str.replace(r'.png$', '')

# Delete wmr.png rows
#read_val = read_val[read_val.ReadValue == wmr]

read_value

In [ ]:
#real_value = pd.read_excel("analog_manual_meter.xlsx", dtype={'actual_value':np.str, 'imageID':np.str})
real_value = pd.read_excel("analog_manual_meter_easy.xlsx", dtype={'actual_value':np.str, 'imageID':np.str})
real_value

In [ ]:
merged = pd.merge(read_value, real_value, on="imageID")
%load_ext google.colab.data_table 
merged

In [ ]:
merged["read_value"] = merged["read_value"].str.replace(r'None', 'N')
merged

In [32]:
merged['IfMatch'] = np.where(merged['read_value'] == merged['actual_value'], 'True', 'False')
merged[merged['IfMatch'] == 'True']
#merged

,imageID,read_value,actual_value,IfMatch
0,10,96220,96220,True
102,10090,20091,20091,True
103,10091,20170,20170,True
136,10120,22083,22083,True
145,10129,22720,22720,True
153,10136,23154,23154,True
165,10147,23791,23791,True
174,10155,24268,24268,True
176,10157,24348,24348,True
182,10162,24627,24627,True


In [35]:
merged["Total"] = merged['read_value'] 

#for row in range(999):
for row in range(499):

  match_counter = 0

  for wmr in range(5):
    if merged['read_value'][row][wmr]== merged['actual_value'][row][wmr]:
      match_counter += 1

  merged['Total'][row] = match_counter

In [37]:
merged["digit_correctness_of_each_meter"] = (merged["Total"]/5)*100
 
#print("accuracy:", (sum (merged["Total"])/ (5*999))*100)
print("accuracy:", (sum (merged["Total"])/ (5*499))*100)

merged

accuracy: 47.414829659318634


,imageID,read_value,actual_value,IfMatch,Total,digit_correctness_of_each_meter
0,10,96220,96220,True,5,100
1,100,20609,00500,False,2,40
2,1000,14365,14366,False,4,80
3,10000,4NNNN,14398,False,0,0
4,10001,14443,14438,False,3,60
...,...,...,...,...,...,...
494,1085,2132N,21734,False,2,40
495,1086,21823,21823,True,5,100
496,1087,61640,21907,False,1,20
497,1088,21992,21997,False,4,80
